<a href="https://colab.research.google.com/github/marchaem/causal-inference/blob/master/Causalinference_Criteo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=Darkred>Causal inference in observational data</font>
---


## <font color=Darkblue>Digital advertising industry.  </font>

> Diemert et al (2018). __A Large Scale Benchmark for Uplift Modeling__, _Proceedings of the AdKDD and TargetAd Workshop_, KDD. London, United Kingdom (August 20). [Data info](https://ailab.criteo.com/criteo-uplift-prediction-dataset/). [Link to the article](https://s3.us-east-2.amazonaws.com/criteo-uplift-dataset/large-scale-benchmark.pdf).

 In the digital advertising industry, the treatment is exposure to different ads and uplift
modeling is used to direct marketing efforts towards users for whom
it is the most efficient. 

To foster research in this topic, [Criteo](https://ailab.criteo.com/) release a publicly available collection of 25 million samples from a
randomized control trial, scaling up previously available datasets
by a healthy 590x factor.



Here we use [`causalinference`](http://causalinferenceinpython.org/index.html): a `python` package that implements various statistical and econometric methods used in the field variously known as Causal Inference, Program Evaluation, or Treatment Effect Analysis.
<br> 

Also check the `causalinference` [vignette](https://github.com/laurencium/causalinference/blob/master/docs/tex/vignette.pdf). Work on [`Causalinference`](http://causalinferenceinpython.org/index.html) started in 2014 by Laurence Wong as a personal side project.




In [0]:
!pip install causalinference

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

<br>

### <font color=Darkred>1. The dataset</font>

The CRITEO-UPLIFT dataset is constructed by assembling data
resulting from several incrementality tests, a particular randomized trial procedure where a random part of the population is prevented from being targeted by advertising. 

The dataset consists of 25M rows, each one representing a user with 12 features, a treatment indicator and 2 binary labels (visits and conversions). 
* Positive labels mean the user visited/converted on the advertiser website during the test period (2 weeks).

* It is usual that advertisers keep only a small control population as it costs them in potential revenue. 

* For privacy reasons the data has been sub-sampled non-uniformly so that the original incrementality level cannot be deduced from the dataset while preserving a realistic, challenging benchmark. 

* Feature names have been anonymized and their values randomly projected so as to keep predictive power while making it practically impossible to recover the original features or user context. 

The dataset is available publicly from the Criteo datasets Web page.

To save time, we are going to use a reduced version of the original dataset with only 10k observations,  with a convertion ratio of around 50% in each group.

__Obs: our working sub-sample is very different than the original proportions__.

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
df=pd.read_csv('Criteo_uplift_raw_reduced.csv', delimiter=',')
print('Number of rows and columns:', df.shape)
print('Data types:', df.dtypes)
df.head(5)

<br>

### <font color=Darkred>2. Descriptive analysis</font>

In [0]:
#Summary statitics
df.describe().round(2).T

In [0]:
# To produce a table with the desc stats of the variables of interest
df[['treatment','exposure','visit','conversion']].groupby('treatment').describe().round(2).T

We could also generate the same table by `visit`. We would see that conversions are perfectly mediated by attrating visitors to the website, as expected.

In [0]:
# Initialize the plot
plt.figure(figsize=(6,6))

# Plots
sns.set(style="whitegrid")
graph1 = sns.barplot(x="treatment", y="conversion", data=df, 
                     capsize=.02, palette="Paired_r")

# Further customize
plt.xlabel('Treatment, Yes = 1, No = 0', size = 'x-large')
plt.ylim(0, 0.6 )
plt.ylabel('Ratio', size = 'x-large') 
plt.title('Convertion rates by study group', 
          loc = "left", size = 'xx-large', pad = 40)
plt.tight_layout();

<br>

### <font color=Darkred>3. Causal inference</font>

In [0]:
# The outcome of interest will be - conversion -
conversion = df.loc[:,"conversion"].to_numpy()
conversion.shape

In [0]:
# Declare the treatement indicator variable
treatment = df.loc[:, "treatment"].to_numpy()
treatment.shape

In [0]:
# Declare the set of covariates
X = df.loc[:, ["f0","f1","f2","f3","f4","f5","f6","f7","f8","f9","f10","f11"]].to_numpy()
X.shape

In [0]:
# Initialize the model
from causalinference import CausalModel
causal = CausalModel(conversion, treatment, X)

print(causal.summary_stats)

This table is just a descriptive summary of the dataset. Similar to what we generated earlier using the `describe`, `bygroup` from `pandas`.

<br>

#### <font color=Darkred>3.1 Ordinary Least Squares (OLS) estimation</font>

In [0]:
causal.est_via_ols()
print(causal.estimates)

<br>

#### <font color=Darkred>3.2 Propensity scoring estimation</font>

To estimate the propensity scores, we run a logistic regression of __treatment__ on the covariates (and tranformations of them) This feature engineering and selection is done automatically by the model.

In [0]:
causal.est_propensity_s()
print(causal.propensity)

To improve the balance between the characteristics of the two groups, we can __trim__ the data by excluding extreme values of the propensity score.

In [0]:
# To trim the data and exclude extreme propensity scores
causal.trim_s()
causal.cutoff

<br>

The optimal cutoff ($\alpha$) suggested by the package is shown above. This means it has dropped observations outside the interval [$\alpha$, 1 - $\alpha$]. Let's show the summary statistics table again to see what happened.

In [0]:
print(causal.summary_stats)

Now that we have the propensity scores, we can __stratify__ the sample into blocks that have units that are more similar in terms of their covariates.

This will make the treatment and the control groups more comparable within each propensity bin, therefore creating a more convincing counterfactual. 

In [0]:
# The default is blocks = 6
causal.stratify_s()
print(causal.strata)

In [0]:
causal.est_via_ols()
causal.est_via_blocking()
causal.est_via_matching(bias_adj=True) ## One-to-one nearest neighbour matching is default

print(causal.estimates)


As we control for the covariate imbalance, the treatment effect converges to zero. 

* This is not the result obtained in the paper using the 25 million observations. But it is good to highlight how sensitive the results are to sampling bias.
* Also, might be due to the fact that I forced the sampling of the 10k observation to have 50% of the examples experiencing a conversion.